In [1]:
# Cell 1: Install dependencies (run only once if not installed)
%pip install librosa scikit-learn pandas numpy xgboost tqdm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Cell 2
import pandas as pd
import os

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df.head()

Train shape: (444, 2)
Test shape: (195, 1)


,filename,label
0,audio_1261.wav,1.0
1,audio_942.wav,1.5
2,audio_1110.wav,1.5
3,audio_1024.wav,1.5
4,audio_538.wav,2.0


In [11]:
# Cell 3 - Updated
def extract_rich_features(audio_path, sr=22050):
    try:
        y, sr = librosa.load(audio_path, sr=sr)
        
        # 1. Trim silence
        y, _ = librosa.effects.trim(y)
        
        # 2. Noise reduction (simple approach via pre-emphasis filter)
        y = librosa.effects.preemphasis(y)
        
        features = []

        # 3. MFCC
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        mfcc_mean = np.mean(mfccs, axis=1)
        mfcc_std = np.std(mfccs, axis=1)
        features.extend(mfcc_mean)
        features.extend(mfcc_std)
        
        # 4. Chroma
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        features.extend(np.mean(chroma, axis=1))

        # 5. Spectral contrast
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        features.extend(np.mean(contrast, axis=1))

        # 6. Tonnetz
        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
        features.extend(np.mean(tonnetz, axis=1))

        # 7. ZCR
        zcr = librosa.feature.zero_crossing_rate(y)
        features.append(np.mean(zcr))

        # 8. RMS energy
        rms = librosa.feature.rms(y=y)
        features.append(np.mean(rms))

        return np.array(features)
    
    except Exception as e:
        print(f"Error with {audio_path}: {e}")
        return np.zeros(58)  # fallback size

In [12]:
# Cell 4 - Reload train features using rich extractor
X_train = []
y_train = []

for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    path = os.path.join(train_audio_folder, row['filename'])
    features = extract_rich_features(path)
    X_train.append(features)
    y_train.append(row['label'])

X_train = np.array(X_train)
y_train = np.array(y_train)

100%|████████████████████████████████████████████████████████████████████████████████| 444/444 [59:11<00:00,  8.00s/it]


In [13]:
# Cell 5 - Test features
X_test = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    path = os.path.join(test_audio_folder, row['filename'])
    features = extract_rich_features(path)
    X_test.append(features)

X_test = np.array(X_test)

100%|████████████████████████████████████████████████████████████████████████████████| 195/195 [15:13<00:00,  4.68s/it]


In [14]:
# Cell 6 - Normalize features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from scipy.stats import pearsonr

# Custom scorer for Pearson
def pearson_corr(y_true, y_pred):
    try:
        return pearsonr(y_true, y_pred)[0]
    except:
        return 0

# Manual K-Fold with proper Pearson check
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for train_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_tr, y_tr)
    
    y_pred = model.predict(X_val)
    score = pearson_corr(y_val, y_pred)
    scores.append(score)

print("Cross-validated Pearson Correlation: ", np.mean(scores))

Cross-validated Pearson Correlation:  0.623287997524514


In [16]:
# Cell 7
predictions = model.predict(X_test)

In [17]:
# Cell 8
submission_df = test_df.copy()
submission_df['label'] = predictions
submission_df.to_csv('submission_new.csv', index=False)
print("Submission file saved as submission.csv")

Submission file saved as submission.csv
